MTG Card Identifier  

In [17]:
# --- Imports & Constants ---
import requests
import pandas as pd
from IPython.display import display

SCRYFALL_API_URL = "https://api.scryfall.com/cards/named"
FRANKFURTER_API_URL = "https://api.frankfurter.app/latest"


In [18]:
def fetch_card_data(card_name: str) -> dict:
    """
    Fetches card data from Scryfall API.
    
    Args:
        card_name (str): Exact name of the MTG card.
        
    Returns:
        dict: Extracted card information.
    """
    params = {"exact": card_name}
    response = requests.get(SCRYFALL_API_URL, params=params)
    response.raise_for_status()
    
    data = response.json()
    return {
        "Name": data.get("name"),
        "Set": data.get("set_name"),
        "Rarity": data.get("rarity"),
        "Oracle Text": data.get("oracle_text"),
        "Image URL": data.get("image_uris", {}).get("normal"),
        "Price USD": data.get("prices", {}).get("usd")
    }


In [19]:
def get_usd_to_eur_rate() -> float:
    """
    Fetches the current USD to EUR exchange rate.
    
    Returns:
        float: USD to EUR rate.
    """
    params = {"from": "USD", "to": "EUR"}
    response = requests.get(FRANKFURTER_API_URL, params=params)
    response.raise_for_status()
    
    data = response.json()
    return data["rates"]["EUR"]


In [20]:
def convert_usd_to_eur(usd_price, rate) -> float | None:
    """
    Converts USD price to EUR using given rate.
    
    Args:
        usd_price (str or float): USD price.
        rate (float): USD to EUR exchange rate.
        
    Returns:
        float or None: Converted EUR price or None if invalid.
    """
    try:
        usd_value = float(usd_price)
        return round(usd_value * rate, 2)
    except (TypeError, ValueError):
        print("⚠️  Warnung: Ungültiger USD-Preis, EUR-Spalte bleibt leer.")
        return None


In [21]:
def save_dataframe_as_csv(df: pd.DataFrame, filename: str) -> None:
    """
    Saves given DataFrame as CSV.
    
    Args:
        df (pd.DataFrame): Data to save.
        filename (str): Destination filename.
    """
    df.to_csv(filename, index=False)
    print(f"✅ CSV gespeichert: {filename}")


In [22]:
def build_card_dataframe(card_info: dict, eur_price: float | None) -> pd.DataFrame:
    """
    Builds a pandas DataFrame from card info and EUR price.
    
    Args:
        card_info (dict): Card data.
        eur_price (float | None): Converted EUR price.
        
    Returns:
        pd.DataFrame: Formatted DataFrame.
    """
    df = pd.DataFrame([card_info])
    df["Price EUR"] = eur_price
    
    # Define column order
    columns_order = ["Name", "Set", "Rarity", "Oracle Text", "Image URL", "Price USD", "Price EUR"]
    df = df[[col for col in columns_order if col in df.columns]]
    
    return df


In [28]:
# --- Multi-Card Pipeline Execution ---

card_names = [
    "traveling chocobo",
    "Lightning Bolt",
    "Counterspell",
    "Black Lotus",
    "Island",
    "Serra Angel"
]

all_card_infos = []

try:
    print(f"🔍 Suche {len(card_names)} Karten...")
    
    # Fetch exchange rate once
    rate = get_usd_to_eur_rate()
    print(f"💱 Aktueller USD→EUR-Kurs: {rate:.4f}\n")
    
    # Process each card
    for card_name in card_names:
        try:
            print(f"→ Hole Karte: '{card_name}'...")
            card_info = fetch_card_data(card_name)
            print(f"✅ '{card_info['Name']}' gefunden.")
            
            eur_price = convert_usd_to_eur(card_info["Price USD"], rate)
            df_card = build_card_dataframe(card_info, eur_price)
            
            all_card_infos.append(df_card)
            
        except requests.HTTPError as e:
            status_code = e.response.status_code if e.response else "N/A"
            print(f"❌ HTTP-Fehler für Karte '{card_name}' ({status_code}): {e}")
        except Exception as e:
            print(f"❌ Unerwarteter Fehler für Karte '{card_name}': {e}")
    
    # Combine all DataFrames
    if all_card_infos:
        df_all_cards = pd.concat(all_card_infos, ignore_index=True)
        
        # Display
        display(df_all_cards)
        
        # Save CSV
        filename = "multi_card_info.csv"
        save_dataframe_as_csv(df_all_cards, filename)
    else:
        print("⚠️  Keine Karten erfolgreich verarbeitet.")
    
except Exception as e:
    print(f"❌ Unerwarteter Fehler in der Pipeline: {e}")


🔍 Suche 6 Karten...
💱 Aktueller USD→EUR-Kurs: 0.8763

→ Hole Karte: 'traveling chocobo'...
✅ 'Traveling Chocobo' gefunden.
→ Hole Karte: 'Lightning Bolt'...
✅ 'Lightning Bolt' gefunden.
→ Hole Karte: 'Counterspell'...
✅ 'Counterspell' gefunden.
→ Hole Karte: 'Black Lotus'...
✅ 'Black Lotus' gefunden.
⚠️  Warnung: Ungültiger USD-Preis, EUR-Spalte bleibt leer.
→ Hole Karte: 'Island'...
✅ 'Island' gefunden.
→ Hole Karte: 'Serra Angel'...
✅ 'Serra Angel' gefunden.


C:\Users\hamud\AppData\Local\Temp\ipykernel_17180\723462414.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_cards = pd.concat(all_card_infos, ignore_index=True)


,Name,Set,Rarity,Oracle Text,Image URL,Price USD,Price EUR
0,Traveling Chocobo,Final Fantasy,mythic,You may look at the top card of your library a...,https://cards.scryfall.io/normal/front/2/4/246...,21.51,18.85
1,Lightning Bolt,Ravnica: Clue Edition,uncommon,Lightning Bolt deals 3 damage to any target.,https://cards.scryfall.io/normal/front/7/7/77c...,0.86,0.75
2,Counterspell,Duskmourn: House of Horror Commander,uncommon,Counter target spell.,https://cards.scryfall.io/normal/front/4/f/4f6...,1.63,1.43
3,Black Lotus,Vintage Masters,bonus,"{T}, Sacrifice this artifact: Add three mana o...",https://cards.scryfall.io/normal/front/b/d/bd8...,None,NaN
4,Island,Tarkir: Dragonstorm,common,({T}: Add {U}.),https://cards.scryfall.io/normal/front/1/f/1ff...,0.04,0.04
5,Serra Angel,Foundations,uncommon,Flying\nVigilance (Attacking doesn't cause thi...,https://cards.scryfall.io/normal/front/3/c/3ce...,0.08,0.07


✅ CSV gespeichert: multi_card_info.csv
